In [16]:
import numpy as np
import xarray as xr
import pandas as pd

import dask
import dask.array as da
from dask import delayed

from matplotlib.path import Path
import matplotlib.ticker as mticker

from glob import glob

from datetime import datetime, timedelta
import os
import wrf
import time

import zarr

In [17]:
siv_folders = glob('/eagle/climate_severe/observations/stageiv/*')
siv_folders.sort()

all_files=[]
for f in siv_folders:
    files_in_folder = glob(f+'/*.nc')
    files_in_folder.sort()
    for fif in files_in_folder:
        all_files.append(fif)
        
geog = xr.open_mfdataset(all_files[0])
dummy_shape = geog.p01m[0].shape #getting the geographic bounds

In [18]:
#%%time

for v in ['p01m']:
    print(v)
    #set zarr storage location
    zarr_store='/eagle/climate_severe/bwallace_scratch/data_request/STAGEIV/STAGEIV'
    #set variables to write to file
    store_vars=[v,'time']

    npList=np.array(all_files)

    time_dims=np.array([1]*(npList.shape[0]*24))
    full_shape=(time_dims.sum(),)+(dummy_shape)
    offsets=np.cumsum(time_dims)
    offsets-=offsets[0]

    #define dictionary of attrs
    ATTRS={
        'time':["time"],
        'lat':["lat"],
        'lon':["lon"],
        v:["time","lat","lon"],

    }

    #store expected shapes of each var in dictionary
    shp={
        'time':(time_dims.sum(),),
        v:full_shape,
    }

    chnks={
        'time':(24,),
        v:(24,)+full_shape[1:],
    }   

    #assign attrs to individual variables
    attrs={}
    attrs[v]={}
    attrs[v]["_ARRAY_DIMENSIONS"]=ATTRS[v]

    attrs['time']={}
    attrs['time']['_ARRAY_DIMENSIONS']=ATTRS['time']
    attrs["time"].update(
        calendar="proleptic_gregorian",units='minutes since 1980-01-01', #make sure this is consistent with function
    )

    #create an empty zarr group 
    group=zarr.group(zarr_store)

    #allocate each variable in the zarr group
    for varname in store_vars:
        v=group.empty(varname,shape=shp[varname],chunks=chnks[varname],dtype='float32',
                      compressor=(zarr.Blosc(cname='zstd', clevel=3)),overwrite=True)
        v.attrs.update(attrs[varname])

print('Done')

p01m
Done
